# 0. Import libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

from statistics import mean

# 1. Functions

In [3]:
# Function to extract sequence features -- BLOSUM62
def BLOSUM62(sequences):
    blosum62 = {
        'A': [4,  -1, -2, -2, 0,  -1, -1, 0, -2,  -1, -1, -1, -1, -2, -1, 1,  0,  -3, -2, 0],  # A
        'R': [-1, 5,  0,  -2, -3, 1,  0,  -2, 0,  -3, -2, 2,  -1, -3, -2, -1, -1, -3, -2, -3], # R
        'N': [-2, 0,  6,  1,  -3, 0,  0,  0,  1,  -3, -3, 0,  -2, -3, -2, 1,  0,  -4, -2, -3], # N
        'D': [-2, -2, 1,  6,  -3, 0,  2,  -1, -1, -3, -4, -1, -3, -3, -1, 0,  -1, -4, -3, -3], # D
        'C': [0,  -3, -3, -3, 9,  -3, -4, -3, -3, -1, -1, -3, -1, -2, -3, -1, -1, -2, -2, -1], # C
        'Q': [-1, 1,  0,  0,  -3, 5,  2,  -2, 0,  -3, -2, 1,  0,  -3, -1, 0,  -1, -2, -1, -2], # Q
        'E': [-1, 0,  0,  2,  -4, 2,  5,  -2, 0,  -3, -3, 1,  -2, -3, -1, 0,  -1, -3, -2, -2], # E
        'G': [0,  -2, 0,  -1, -3, -2, -2, 6,  -2, -4, -4, -2, -3, -3, -2, 0,  -2, -2, -3, -3], # G
        'H': [-2, 0,  1,  -1, -3, 0,  0,  -2, 8,  -3, -3, -1, -2, -1, -2, -1, -2, -2, 2,  -3], # H
        'I': [-1, -3, -3, -3, -1, -3, -3, -4, -3, 4,  2, -3, 1,  0, -3, -2, -1, -3, -1, 3],  # I
        'L': [-1, -2, -3, -4, -1, -2, -3, -4, -3, 2,  4,  -2, 2,  0, -3, -2, -1, -2, -1, 1],  # L
        'K': [-1, 2,  0,  -1, -3, 1,  1,  -2, -1, -3, -2, 5,  -1, -3, -1, 0,  -1, -3, -2, -2], # K
        'M': [-1, -1, -2, -3, -1, 0,  -2, -3, -2, 1,  2,  -1, 5,  0, -2, -1, -1, -1, -1, 1],  # M
        'F': [-2, -3, -3, -3, -2, -3, -3, -3, -1, 0,  0,  -3, 0,  6, -4, -2, -2, 1,  3,  -1], # F
        'P': [-1, -2, -2, -1, -3, -1, -1, -2, -2, -3, -3, -1, -2, -4, 7,  -1, -1, -4, -3, -2], # P
        'S': [1,  -1, 1,  0,  -1, 0,  0,  0,  -1, -2, -2, 0,  -1, -2, -1, 4,  1,  -3, -2, -2], # S
        'T': [0,  -1, 0,  -1, -1, -1, -1, -2, -2, -1, -1, -1, -1, -2, -1, 1,  5,  -2, -2, 0],  # T
        'W': [-3, -3, -4, -4, -2, -2, -3, -2, -2, -3, -2, -3, -1, 1, -4, -3, -2, 11, 2,  -3], # W
        'Y': [-2, -2, -2, -3, -2, -1, -2, -3, 2,  -1, -1, -2, -1, 3, -3, -2, -2, 2,  7,  -1], # Y
        'V': [0,  -3, -3, -3, -1, -2, -2, -3, -3, 3,  1,  -2, 1,  -1, -2, -2, 0,  -3, -1, 4],  # V
        '*': [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],  # *
        'X': [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],  # X
        'U': [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],  # U
        '_': [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],  # _
    }
    encodings = []
    for sequence in sequences:
        code=[]  
        for j in sequence:
            code = code + blosum62[j]
        encodings.append(np.array(code))       
    return encodings

# Function to select negative data based on the composition of positive data
def Negative(df_neg_ST, df_neg_Y, positive):
    
    # The number of ST/Y sites is consistent in positive and negative data
    if "Y" in positive["sequence"].str[7].value_counts().keys():
        num_Y = positive["sequence"].str[7].value_counts()["Y"]
    else:
        num_Y = 0
    num_ST = len(positive)- num_Y

    negative_ST = df_neg_ST.sample(n=num_ST).reset_index(drop=True)
    negative_Y = df_neg_Y.sample(n=num_Y).reset_index(drop=True)
    negative = pd.concat([negative_ST, negative_Y], axis=0).reset_index(drop=True).sample(frac=1).reset_index(drop=True)

    negative["label"]=[0]*len(negative)

    # Combine protein- and sequence-related negative data
    negative = negative.rename(columns={'protein neg': 'SubID', 'sequence neg': 'sequence'}) 
    
    return negative

# 2. Read data

### 2.1 Read understudied kinases and their highly similar kinases

In [4]:
# AGC group as an example, add kinases in other groups to train and test more models
dict_N = {
 'P22694': ['P17612'],
 'O75676': ['O75582', 'Q15349', 'P51812'],
 'Q96BR1': ['O00141'], 
 'Q9Y243': ['P31751', 'P31749'],
 'Q15208': ['Q9NRM7', 'O95835', 'Q9Y2H1'],
 'Q9UBS0': ['P23443', 'O75676', 'O75582', 'P51812', 'Q15349', 'Q15418']
}


### 2.2 Read human K-S data and negative data

In [5]:
df = pd.read_csv("../0_data/Human K_S data.csv")

In [6]:
df_neg_Y = pd.read_csv("../0_data/negative Y sites.csv")
df_neg_ST = pd.read_csv("../0_data/negative ST sites.csv")

# 3. SVM model 

In [7]:
KIN =[]
K_accuracy2 = []
K_precision2 = []
K_recall2 = []
K_F1score2 = []
K_AUC2 = []

len_pos = []


for k in dict_N.keys():
    
    # Define empty lists to store performance metrics 
    Accuracy2 = []
    Precision2 = []
    Recall2 = []
    F1_score2 = []
    AUC2 = []

    # 3.1.1 Positive training data --- psites of highly similar kinases
    df1 = df[df["KIN_ACC"].isin(dict_N[k])].reset_index(drop=True)
    positive = df1[["SUB_ACC","PEPTIDE"]].reset_index(drop=True)
    positive["label"] = [1]*len(positive)
    positive = positive.rename(columns={'SUB_ACC': 'SubID', 'PEPTIDE': 'sequence'}) 
    
    
    # 3.1.2 Validation data --- psites of understudied kinases
    df_val = df[df["KIN_ACC"] == k].reset_index(drop=True)
    val = df_val[["SUB_ACC","PEPTIDE"]].reset_index(drop=True)
    val["label"] = [1]*len(val)
    val=val[val["PEPTIDE"].str.len() == 15].reset_index(drop=True)
    val_positive = val.rename(columns={'SUB_ACC': 'SubID', 'PEPTIDE': 'sequence'})
        

    # For each understudied kinase, the process is repeated 10 times to get average performance metrics 
    for i in range(0, 10):
        
        # 3.1.3 Negative training data --- different every time
        negative = Negative(df_neg_ST, df_neg_Y, positive)
        
        val_negative = Negative(df_neg_ST, df_neg_Y, val_positive)
        
        
        # 3.1.4 Train and test splits
        df_input = pd.concat([positive, negative], axis=0).reset_index(drop=True)
        df_input = df_input[df_input["sequence"].str.len() == 15].reset_index(drop=True)
        train, test = train_test_split(df_input, test_size=0.2, random_state=42, shuffle=True)
        train = train.reset_index(drop=True)
        test = test.reset_index(drop=True)
        
        df_input_val = pd.concat([val_positive, val_negative], axis=0).reset_index(drop=True)
        validate = df_input_val.sample(frac=1).reset_index(drop=True)

        

        # 3.2 Data encoding
        # 3.2.1 Sequence data
        X_seq_train = np.array(BLOSUM62(train["sequence"]))
        X_seq_test = np.array(BLOSUM62(test["sequence"]))
        X_seq_validate = np.array(BLOSUM62(validate["sequence"]))

        Y_train = np.array(train["label"])
        Y_test = np.array(test["label"])
        Y_validate = np.array(validate["label"])

#         X_seq_tr = np.array(np.concatenate(X_seq_train).flat).reshape(len(X_seq_train), len(X_seq_train[0]))
#         X_seq_te = np.array(np.concatenate(X_seq_test).flat).reshape(len(X_seq_test), len(X_seq_test[0]))
#         X_seq_val = np.array(np.concatenate(X_seq_validate).flat).reshape(len(X_seq_validate), len(X_seq_validate[0]))

        
        #=======================
        # 3.3 SVM model
        SVM_clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
        SVM_clf.fit(X_seq_train, Y_train)
        preds = SVM_clf.predict(X_seq_test)

        pred2 = SVM_clf.predict(X_seq_validate)

        target_names = ['neg 0', 'pos 1']
        report2 = classification_report(Y_validate, pred2, target_names=target_names, output_dict=True)
        
        acc2 = round(report2["accuracy"],3)
        pre2 = round(report2["weighted avg"]["precision"],3)
        recall2 = round(report2["weighted avg"]["recall"],3)
        f12 = round(report2["weighted avg"]["f1-score"],3)
        auc2 = roc_auc_score(Y_validate, pred2, average='weighted')

        Accuracy2.append(acc2)
        Precision2.append(pre2)
        Recall2.append(recall2)
        F1_score2.append(f12)
        AUC2.append(auc2)
        
        
        
        
    KIN.append(k)

    K_accuracy2.append(mean(Accuracy2))
    K_precision2.append(mean(Precision2))
    K_recall2.append(mean(Recall2))
    K_F1score2.append(mean(F1_score2))
    K_AUC2.append(mean(AUC2))

    len_pos.append(len(positive))

In [8]:
X_seq_train.shape

(528, 300)

# 4. Results

In [9]:
gene={}
for i in range(0, len(df)):
    gene[df["KIN_ACC"][i]] = df["KINASE"][i]

G={}
for i in range(0, len(df)):
    G[df["KIN_ACC"][i]] = df["Group"][i]
    
gene_name=[]
group_name=[]
num_of_test=[]
for k in dict_N.keys():
    gene_name.append(gene[k])
    group_name.append(G[k])
    num_of_test.append(len(df[df["KIN_ACC"] == k].drop_duplicates(subset = ["PEPTIDE"])))
    

In [10]:
results = pd.DataFrame()

results["kinase"] = KIN
results["gene names"] = gene_name
results["group"] = group_name
results["test size"]=num_of_test

results["Acc svm"] = K_accuracy2
results["Pre svm"] = K_precision2
results["Recall svm"] = K_recall2
results["F1 score svm"] = K_F1score2
results["AUC svm"] = K_AUC2

results["No of positive sites"] = len_pos


In [11]:
results.sort_values(by=["test size"]) # 32--4

,kinase,gene names,group,test size,Acc svm,Pre svm,Recall svm,F1 score svm,AUC svm,No of positive sites
0,P22694,PRKACB,AGC,6,0.9002,0.9098,0.9002,0.8994,0.900000,1031
1,O75676,RPS6KA4,AGC,6,0.9500,0.9584,0.9500,0.9495,0.950000,142
2,Q96BR1,SGK3,AGC,6,0.7250,0.7799,0.7250,0.7124,0.725000,89
5,Q9UBS0,RPS6KB2,AGC,9,0.9331,0.9363,0.9331,0.9331,0.933333,330
4,Q15208,STK38,AGC,10,0.9000,0.9170,0.9000,0.8990,0.900000,56
3,Q9Y243,AKT3,AGC,11,0.8728,0.8809,0.8728,0.8719,0.872727,528
